In [1]:
# 附加题中，我们选择使用cifar10
import paddle
from paddle.vision.transforms import Normalize, Compose, Transpose
from paddle.vision.datasets import Cifar10
def get_cifar10_dataloader():
    # cifar10是一个三通道的图片
    transforms = Compose([Normalize(mean=[127.5, 127.5, 127.5], std= [127.5, 127.5, 127.5], data_format='HWC'), Transpose() ])
    # 数据集
    train_datasets = Cifar10(mode='train', transform=transforms)
    test_datasets = Cifar10(mode='test', transform=transforms)
    # 数据读取器
    train_loader = paddle.io.DataLoader(train_datasets, batch_size=64, shuffle = True, num_workers=1, drop_last=True)
    test_loader = paddle.io.DataLoader(test_datasets, batch_size=64, shuffle = False, num_workers=1, drop_last=False)
    return train_loader, test_loader

In [2]:
train_loader, test_loader = get_cifar10_dataloader()

e:\SoftWare\Program\Anaconda\envs\d2l\lib\site-packages\paddle\io\reader.py:433: UserWarning: DataLoader with multi-process mode is not supported on MacOs and Windows currently. Please use signle-process mode with num_workers = 0 instead
  warnings.warn(


In [3]:
import visualdl
import visualdl.server.app
visualdl.server.app.run('./run/cifar_log',
                        host = "127.0.0.1",
                        port=8081,
                        cache_timeout=5)

17988

In [4]:
# 定义模型结构
# 首先只来一层卷积

import paddle.nn.functional as F
from paddle.nn import Conv2D, MaxPool2D, Linear

class Cifar_ONE_CONV(paddle.nn.Layer):
    def __init__(self):
        super(Cifar_ONE_CONV, self).__init__()

        # 定义一层卷积层
        self.conv1 = Conv2D(in_channels=3, out_channels=20, kernel_size=5, stride=1, padding=2)
        # 定义最大池化
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        # 这样输出的就是20*16*16,定义一个全连接
        self.fc = Linear(in_features=5120, out_features=10)

    def forward(self, inputs, label):
        x = self.conv1(inputs)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = paddle.reshape(x, [x.shape[0], -1])
        x = self.fc(x)

        if label is not None:
            acc = paddle.metric.accuracy(input=x, label = label)
            return x, acc
        else:
            return x
use_gpu = True
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')

Place(gpu:0)

In [5]:
# 定义训练过程
from visualdl import LogWriter
logwriter = LogWriter(logdir="./run/cifar_log/one_conv")
def train_Cifar_ONE(model):
    # 定义优化器
    opt = paddle.optimizer.Adam(learning_rate=0.01, parameters=model.parameters())

    # 训练参数
    EPOCH_NUM = 10
    iter = 0
    iter2 = 0
    for epoch_id in range(EPOCH_NUM):
        total_test_accuracy = 0  # 用于累积测试准确率
        for batch_id, data in enumerate(train_loader()):
            model.train()
            # 数据准备
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels, (-1,1))
            labels = paddle.to_tensor(labels)

            # 前向
            predict, acc = model(images, labels)

            # 损失
            loss = F.cross_entropy(predict, labels)
            avg_loss = paddle.mean(loss)

            # 可以记录
            if batch_id%150 ==  0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(), acc.numpy()))
                logwriter.add_scalar(tag = 'train/acc', step = iter, value=acc.numpy())
                logwriter.add_scalar(tag = 'train/loss' , step = iter, value=avg_loss.numpy())
                iter+=150
            # 反向传播
            avg_loss.backward()
            opt.step()
            opt.clear_grad()

        for batch_id, data in enumerate(test_loader()):
            model.eval()
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels,(-1,1))
            labels = paddle.to_tensor(labels)

            # 预测
            predicts, acc = model(images, labels)
            loss = F.cross_entropy(predicts, labels)
            avg_loss= paddle.mean(loss)

            total_test_accuracy += acc.numpy()

            if batch_id % 100 == 0:
                logwriter.add_scalar(tag='test/acc', step=iter2, value=acc.numpy())
                logwriter.add_scalar(tag='test/loss', step=iter2, value=avg_loss.numpy())
                iter2+=100
        
        total_test_accuracy /= len(test_loader())
        logwriter.add_scalar(tag='test/total_acc', step=epoch_id, value=total_test_accuracy)
            

                

In [6]:
model_cifar_one = Cifar_ONE_CONV()
train_Cifar_ONE(model_cifar_one)

epoch: 0, batch: 0, loss is: [2.591126], acc is [0.0625]
epoch: 0, batch: 150, loss is: [1.3860409], acc is [0.5]
epoch: 0, batch: 300, loss is: [1.6586318], acc is [0.421875]
epoch: 0, batch: 450, loss is: [1.3182759], acc is [0.546875]
epoch: 0, batch: 600, loss is: [1.3328769], acc is [0.5]
epoch: 0, batch: 750, loss is: [1.2842821], acc is [0.5]
epoch: 1, batch: 0, loss is: [1.5553687], acc is [0.4375]
epoch: 1, batch: 150, loss is: [1.4089432], acc is [0.46875]
epoch: 1, batch: 300, loss is: [1.7016492], acc is [0.40625]
epoch: 1, batch: 450, loss is: [1.7483552], acc is [0.4375]
epoch: 1, batch: 600, loss is: [1.9843371], acc is [0.484375]
epoch: 1, batch: 750, loss is: [1.5778127], acc is [0.59375]
epoch: 2, batch: 0, loss is: [0.99006325], acc is [0.671875]
epoch: 2, batch: 150, loss is: [1.0828569], acc is [0.671875]
epoch: 2, batch: 300, loss is: [1.5346546], acc is [0.515625]
epoch: 2, batch: 450, loss is: [1.424325], acc is [0.546875]
epoch: 2, batch: 600, loss is: [1.14808

In [7]:
# 定义模型结构

import paddle.nn.functional as F
from paddle.nn import Conv2D, MaxPool2D, Linear

class Cifar_TWO_CONV(paddle.nn.Layer):
    def __init__(self):
        super(Cifar_TWO_CONV, self).__init__()

        # 定义一层卷积层
        self.conv1 = Conv2D(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        # 定义最大池化
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        # 定义第二层卷积层
        self.conv2 = Conv2D(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=2)
        # 定义最大池化
        self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
        # 这样输出的就是20*16*16,定义一个全连接
        self.fc = Linear(in_features=1536, out_features=10)

    def forward(self, inputs, label):
        x = self.conv1(inputs)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = paddle.reshape(x, [x.shape[0], -1])
        x = self.fc(x)

        if label is not None:
            acc = paddle.metric.accuracy(input=x, label = label)
            return x, acc
        else:
            return x
use_gpu = True
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')

Place(gpu:0)

In [8]:
# 定义训练过程
from visualdl import LogWriter
logwriter = LogWriter(logdir="./run/cifar_log/two_conv")
def train_Cifar_TWO(model):
    # 定义优化器
    opt = paddle.optimizer.Adam(learning_rate=0.01, parameters=model.parameters())

    # 训练参数
    EPOCH_NUM = 10  
    iter = 0
    iter2 = 0
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            model.train()
            # 数据准备
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels, (-1,1))
            labels = paddle.to_tensor(labels)

            # 前向
            predict, acc = model(images, labels)

            # 损失
            loss = F.cross_entropy(predict, labels)
            avg_loss = paddle.mean(loss)

            # 可以记录
            if batch_id%150 ==  0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(), acc.numpy()))
                logwriter.add_scalar(tag = 'train/acc', step = iter, value=acc.numpy())
                logwriter.add_scalar(tag = 'train/loss' , step = iter, value=avg_loss.numpy())
                iter+=150
            # 反向传播
            avg_loss.backward()
            opt.step()
            opt.clear_grad()
        total_test_accuracy=0
        for batch_id, data in enumerate(test_loader()):
            model.eval()
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels,(-1,1))
            labels = paddle.to_tensor(labels)

            # 预测
            predicts, acc = model(images, labels)
            loss = F.cross_entropy(predicts, labels)
            avg_loss= paddle.mean(loss)

            total_test_accuracy += acc.numpy()

            if batch_id % 100 == 0:
                logwriter.add_scalar(tag='test/acc', step=iter2, value=acc.numpy())
                logwriter.add_scalar(tag='test/loss', step=iter2, value=avg_loss.numpy())
                iter2+=100
        
        total_test_accuracy /= len(test_loader())
        logwriter.add_scalar(tag='test/total_acc', step=epoch_id, value=total_test_accuracy)

                

In [9]:
model_cifar_two = Cifar_TWO_CONV()
train_Cifar_TWO(model_cifar_two)

epoch: 0, batch: 0, loss is: [2.779589], acc is [0.140625]
epoch: 0, batch: 150, loss is: [2.3006978], acc is [0.09375]
epoch: 0, batch: 300, loss is: [2.3085442], acc is [0.0625]
epoch: 0, batch: 450, loss is: [2.2996986], acc is [0.140625]
epoch: 0, batch: 600, loss is: [2.3036814], acc is [0.078125]
epoch: 0, batch: 750, loss is: [2.3033], acc is [0.046875]
epoch: 1, batch: 0, loss is: [2.3024077], acc is [0.1875]
epoch: 1, batch: 150, loss is: [2.2998648], acc is [0.125]
epoch: 1, batch: 300, loss is: [2.3046794], acc is [0.140625]
epoch: 1, batch: 450, loss is: [2.3101408], acc is [0.140625]
epoch: 1, batch: 600, loss is: [2.3043513], acc is [0.09375]
epoch: 1, batch: 750, loss is: [2.2676044], acc is [0.09375]
epoch: 2, batch: 0, loss is: [2.157767], acc is [0.1875]
epoch: 2, batch: 150, loss is: [2.1264591], acc is [0.234375]
epoch: 2, batch: 300, loss is: [1.9482131], acc is [0.296875]
epoch: 2, batch: 450, loss is: [1.9311458], acc is [0.328125]
epoch: 2, batch: 600, loss is: 

In [10]:
# 定义模型结构
# 首先只来一层卷积

import paddle.nn.functional as F
from paddle.nn import Conv2D, MaxPool2D, Linear

class Cifar_THree_CONV2(paddle.nn.Layer):
    def __init__(self):
        super(Cifar_THree_CONV2, self).__init__()

        # 定义一层卷积层
        self.conv1 = Conv2D(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        # 定义最大池化
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        # 定义第二层卷积层
        self.conv2 = Conv2D(in_channels=12, out_channels=20, kernel_size=5, stride=1, padding=2)
        # 定义最大池化
        self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
        # 再定义一层卷积层
        self.conv3 = Conv2D(in_channels=20, out_channels=32,kernel_size=3, stride=1, padding=1)
        self.max_pool3 = MaxPool2D(kernel_size=2,stride=2)
        # 这样输出的就是20*16*16,定义一个全连接
        self.fc = Linear(in_features=2048, out_features=10)

    def forward(self, inputs, label):
        x = self.conv1(inputs)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = paddle.reshape(x, [x.shape[0], -1])
        x = self.fc(x)

        if label is not None:
            acc = paddle.metric.accuracy(input=x, label = label)
            return x, acc
        else:
            return x
use_gpu = True
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')

Place(gpu:0)

In [12]:
# 定义训练过程
from visualdl import LogWriter
logwriter = LogWriter(logdir="./run/cifar_log/three_conv")
def train_Cifar_Three(model):
    # 定义优化器
    opt = paddle.optimizer.Adam(learning_rate=0.01, parameters=model.parameters())

    # 训练参数
    EPOCH_NUM = 10
    iter = 0
    iter2 = 0
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            model.train()
            # 数据准备
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels, (-1,1))
            labels = paddle.to_tensor(labels)

            # 前向
            predict, acc = model(images, labels)

            # 损失
            loss = F.cross_entropy(predict, labels)
            avg_loss = paddle.mean(loss)

            # 可以记录
            if batch_id%150 ==  0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(), acc.numpy()))
                logwriter.add_scalar(tag = 'train/acc', step = iter, value=acc.numpy())
                logwriter.add_scalar(tag = 'train/loss' , step = iter, value=avg_loss.numpy())
                iter+=150
            # 反向传播
            avg_loss.backward()
            opt.step()
            opt.clear_grad()

        total_test_accuracy=0
        for batch_id, data in enumerate(test_loader()):
            model.eval()
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels,(-1,1))
            labels = paddle.to_tensor(labels)

            # 预测
            predicts, acc = model(images, labels)
            loss = F.cross_entropy(predicts, labels)
            avg_loss= paddle.mean(loss)

            total_test_accuracy += acc.numpy()

            if batch_id % 100 == 0:
                logwriter.add_scalar(tag='test/acc', step=iter2, value=acc.numpy())
                logwriter.add_scalar(tag='test/loss', step=iter2, value=avg_loss.numpy())
                iter2+=100
        
        total_test_accuracy /= len(test_loader())
        logwriter.add_scalar(tag='test/total_acc', step=epoch_id, value=total_test_accuracy)
                

In [13]:
model_cifar_two2 = Cifar_THree_CONV2()
train_Cifar_Three(model_cifar_two2)


epoch: 0, batch: 0, loss is: [2.321579], acc is [0.109375]
epoch: 0, batch: 150, loss is: [1.4878459], acc is [0.5]
epoch: 0, batch: 300, loss is: [1.502988], acc is [0.375]
epoch: 0, batch: 450, loss is: [1.3144822], acc is [0.53125]
epoch: 0, batch: 600, loss is: [1.5574405], acc is [0.46875]
epoch: 0, batch: 750, loss is: [1.1929733], acc is [0.609375]
epoch: 1, batch: 0, loss is: [1.4715104], acc is [0.5]
epoch: 1, batch: 150, loss is: [1.2889777], acc is [0.515625]
epoch: 1, batch: 300, loss is: [1.548099], acc is [0.484375]
epoch: 1, batch: 450, loss is: [1.4900177], acc is [0.46875]
epoch: 1, batch: 600, loss is: [1.1400313], acc is [0.59375]
epoch: 1, batch: 750, loss is: [1.6508226], acc is [0.40625]
epoch: 2, batch: 0, loss is: [1.4976201], acc is [0.4375]
epoch: 2, batch: 150, loss is: [1.3817183], acc is [0.5]
epoch: 2, batch: 300, loss is: [1.3742287], acc is [0.453125]
epoch: 2, batch: 450, loss is: [1.5464451], acc is [0.5625]
epoch: 2, batch: 600, loss is: [1.3242497], 

In [14]:
# 定义模型结构
# 使用dropout

import paddle.nn.functional as F
from paddle.nn import Conv2D, MaxPool2D, Linear

class Cifar_TWO_CONV2_drop(paddle.nn.Layer):
    def __init__(self):
        super(Cifar_TWO_CONV2_drop, self).__init__()

        # 定义一层卷积层
        self.conv1 = Conv2D(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        # 定义最大池化
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        # 定义第二层卷积层
        self.conv2 = Conv2D(in_channels=12, out_channels=20, kernel_size=5, stride=1, padding=2)
        # 定义最大池化
        self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
        # 再定义一层卷积层
        self.conv3 = Conv2D(in_channels=20, out_channels=32,kernel_size=5, stride=1, padding=2)
        # 这样输出的就是20*16*16,定义一个全连接
        self.fc = Linear(in_features=2048, out_features=3072)
        self.fc2 = Linear(in_features=3072,out_features=1024)
        self.fc3 = Linear(in_features=1024, out_features=10)
    def forward(self, inputs, label, mode='upscale_in_train'):
        x = self.conv1(inputs)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = self.conv3(x)
        x = paddle.reshape(x, [x.shape[0], -1])
        x = self.fc(x)
        x = F.dropout(x, p = 0.007,mode=mode)
        x = self.fc2(x)
        x = F.dropout(x, p = 0.005, mode=mode)
        x = self.fc3(x)
        if label is not None:
            acc = paddle.metric.accuracy(input=x, label = label)
            return x, acc
        else:
            return x
use_gpu = True
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')

Place(gpu:0)

In [15]:
# 定义训练过程
from visualdl import LogWriter
from paddle.optimizer.lr import CosineAnnealingDecay
logwriter = LogWriter(logdir="./run/cifar_log/three_conv_drop")
def train_Cifar_TWO2_drop(model):
    # 定义优化器
    EPOCH_NUM = 15
    opt = paddle.optimizer.Adam(learning_rate=0.01, parameters=model.parameters())

    # 训练参数
    iter = 0
    iter2 = 0
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            model.train()
            # 数据准备
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels, (-1,1))
            labels = paddle.to_tensor(labels)

            # 前向
            predict, acc = model(images, labels)

            # 损失
            loss = F.cross_entropy(predict, labels)
            avg_loss = paddle.mean(loss)

            # 可以记录
            if batch_id%150 ==  0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(), acc.numpy()))
                logwriter.add_scalar(tag = 'train/acc', step = iter, value=acc.numpy())
                logwriter.add_scalar(tag = 'train/loss' , step = iter, value=avg_loss.numpy())
                iter+=150
            # 反向传播
            avg_loss.backward()
            opt.step()
            opt.clear_grad()

        total_test_accuracy=0
        for batch_id, data in enumerate(test_loader()):
            model.eval()
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels,(-1,1))
            labels = paddle.to_tensor(labels)

            # 预测
            predicts, acc = model(images, labels)
            loss = F.cross_entropy(predicts, labels)
            avg_loss= paddle.mean(loss)

            total_test_accuracy += acc.numpy()

            if batch_id % 100 == 0:
                logwriter.add_scalar(tag='test/acc', step=iter2, value=acc.numpy())
                logwriter.add_scalar(tag='test/loss', step=iter2, value=avg_loss.numpy())
                iter2+=100
        
        total_test_accuracy /= len(test_loader())
        logwriter.add_scalar(tag='test/total_acc', step=epoch_id, value=total_test_accuracy)

                

In [16]:
model_cifar_drop = Cifar_TWO_CONV2_drop()
train_Cifar_TWO2_drop(model_cifar_drop)

epoch: 0, batch: 0, loss is: [2.6706896], acc is [0.109375]
epoch: 0, batch: 150, loss is: [2.399619], acc is [0.078125]
epoch: 0, batch: 300, loss is: [2.36034], acc is [0.09375]
epoch: 0, batch: 450, loss is: [2.3338962], acc is [0.078125]
epoch: 0, batch: 600, loss is: [2.2996335], acc is [0.078125]
epoch: 0, batch: 750, loss is: [2.305578], acc is [0.109375]
epoch: 1, batch: 0, loss is: [2.2970946], acc is [0.078125]
epoch: 1, batch: 150, loss is: [2.4451375], acc is [0.078125]
epoch: 1, batch: 300, loss is: [2.3126059], acc is [0.140625]
epoch: 1, batch: 450, loss is: [2.3647337], acc is [0.109375]
epoch: 1, batch: 600, loss is: [2.357091], acc is [0.0625]
epoch: 1, batch: 750, loss is: [2.3048577], acc is [0.140625]
epoch: 2, batch: 0, loss is: [2.3547986], acc is [0.109375]
epoch: 2, batch: 150, loss is: [2.3652806], acc is [0.171875]
epoch: 2, batch: 300, loss is: [2.314753], acc is [0.09375]
epoch: 2, batch: 450, loss is: [3262.0986], acc is [0.0625]
epoch: 2, batch: 600, loss

In [17]:
# 定义模型结构

import paddle.nn.functional as F
from paddle.nn import Conv2D, MaxPool2D, Linear, BatchNorm2D

class Cifar_TWO_CONV2_norm(paddle.nn.Layer):
    def __init__(self):
        super(Cifar_TWO_CONV2_norm, self).__init__()

        # 定义一层卷积层
        self.conv1 = Conv2D(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        # 定义批归一化层
        self.bn1 = BatchNorm2D(12)
        # 定义最大池化
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        # 定义第二层卷积层
        self.conv2 = Conv2D(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=2)
        # 第二个批归一化层
        self.bn2 = BatchNorm2D(24)
        # 定义最大池化
        self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
        # 定义第三个卷积层
        self.conv3 = Conv2D(in_channels=24, out_channels= 32, kernel_size=3, stride=1, padding=1)
        self.bn3 = BatchNorm2D(32)
        self.max_pool3 = MaxPool2D(kernel_size=2, stride=2)
        # 这样输出的就是32*8*8,定义一个全连接
        self.fc = Linear(in_features=512, out_features=10)

    def forward(self, inputs, label, mode='upscale_in_train'):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.max_pool3(x)
        x = paddle.reshape(x, [x.shape[0], -1])
        x = self.fc(x)

        x = F.dropout(x, p = 0.01,mode=mode)
        if label is not None:
            acc = paddle.metric.accuracy(input=x, label = label)
            return x, acc
        else:
            return x
use_gpu = True
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')

Place(gpu:0)

In [19]:
# 定义训练过程
from visualdl import LogWriter
from paddle.optimizer.lr import CosineAnnealingDecay
from paddle.nn import BatchNorm2D
logwriter = LogWriter(logdir="./run/cifar_log/two_conv_new_batnorm")
def train_Cifar_TWO2_norm(model):
    # 定义优化器
    EPOCH_NUM = 10
    opt = paddle.optimizer.Adam(learning_rate=0.01, weight_decay=paddle.regularizer.L2Decay(coeff=1e-5),parameters=model.parameters())

    # 训练参数
    iter = 0
    iter2 = 0
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            model.train()
            # 数据准备
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels, (-1,1))
            labels = paddle.to_tensor(labels)

            # 前向
            predict, acc = model(images, labels)

            # 损失
            loss = F.cross_entropy(predict, labels)
            avg_loss = paddle.mean(loss)

            # 可以记录
            if batch_id%150 ==  0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(), acc.numpy()))
                logwriter.add_scalar(tag = 'train/acc', step = iter, value=acc.numpy())
                logwriter.add_scalar(tag = 'train/loss' , step = iter, value=avg_loss.numpy())
                iter+=150
            # 反向传播
            avg_loss.backward()
            opt.step()
            opt.clear_grad()

        total_test_accuracy=0
        for batch_id, data in enumerate(test_loader()):
            model.eval()
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels,(-1,1))
            labels = paddle.to_tensor(labels)

            # 预测
            predicts, acc = model(images, labels)
            loss = F.cross_entropy(predicts, labels)
            avg_loss= paddle.mean(loss)

            total_test_accuracy += acc.numpy()

            if batch_id % 100 == 0:
                logwriter.add_scalar(tag='test/acc', step=iter2, value=acc.numpy())
                logwriter.add_scalar(tag='test/loss', step=iter2, value=avg_loss.numpy())
                iter2+=100
        
        total_test_accuracy /= len(test_loader())
        logwriter.add_scalar(tag='test/total_acc', step=epoch_id, value=total_test_accuracy)
                

In [20]:
model = Cifar_TWO_CONV2_norm()
train_Cifar_TWO2_norm(model)

e:\SoftWare\Program\Anaconda\envs\d2l\lib\site-packages\paddle\nn\layer\norm.py:777: UserWarning: When training, we now always track global mean and variance.
  warnings.warn(


epoch: 0, batch: 0, loss is: [2.8470454], acc is [0.0625]
epoch: 0, batch: 150, loss is: [1.3579888], acc is [0.46875]
epoch: 0, batch: 300, loss is: [1.173275], acc is [0.546875]
epoch: 0, batch: 450, loss is: [1.1697747], acc is [0.625]
epoch: 0, batch: 600, loss is: [1.308835], acc is [0.5625]
epoch: 0, batch: 750, loss is: [1.1410599], acc is [0.53125]
epoch: 1, batch: 0, loss is: [1.1541077], acc is [0.59375]
epoch: 1, batch: 150, loss is: [1.0717705], acc is [0.640625]
epoch: 1, batch: 300, loss is: [1.142252], acc is [0.640625]
epoch: 1, batch: 450, loss is: [1.0128442], acc is [0.609375]
epoch: 1, batch: 600, loss is: [0.9101277], acc is [0.765625]
epoch: 1, batch: 750, loss is: [0.94764787], acc is [0.671875]
epoch: 2, batch: 0, loss is: [0.8985741], acc is [0.71875]
epoch: 2, batch: 150, loss is: [0.921988], acc is [0.65625]
epoch: 2, batch: 300, loss is: [0.7929771], acc is [0.75]
epoch: 2, batch: 450, loss is: [0.8885379], acc is [0.703125]
epoch: 2, batch: 600, loss is: [0

In [21]:
# 定义模型结构
# 使用dropout

import paddle.nn.functional as F
from paddle.nn import Conv2D, MaxPool2D, Linear, BatchNorm2D

class Cifar_TWO_CONV2_norm_end(paddle.nn.Layer):
    def __init__(self):
        super(Cifar_TWO_CONV2_norm_end, self).__init__()

        # 定义一层卷积层
        self.conv1 = Conv2D(in_channels=3, out_channels=12, kernel_size=3, stride=1, padding=1)
        # 定义批归一化层
        self.bn1 = BatchNorm2D(12)
        # 定义最大池化
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        # 定义第二层卷积层
        self.conv2 = Conv2D(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=2)
        # 第二个批归一化层
        self.bn2 = BatchNorm2D(24)
        # 定义最大池化
        self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
        # 定义第三个卷积层
        self.conv3 = Conv2D(in_channels=24, out_channels= 32, kernel_size=3, stride=1, padding=1)
        self.bn3 = BatchNorm2D(32)
        self.max_pool3 = MaxPool2D(kernel_size=2, stride=2)
        # 这样输出的就是32*8*8,定义一个全连接
        self.fc = Linear(in_features=512, out_features=218)
        self.fc2 = Linear(in_features=218,out_features=10)

    def forward(self, inputs, label, mode='upscale_in_train'):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = self.max_pool3(x)
        x = paddle.reshape(x, [x.shape[0], -1])
        x = self.fc(x)
        x = F.dropout(x)
        x = self.fc2(x)

        x = F.dropout(x, p = 0.01,mode=mode)
        if label is not None:
            acc = paddle.metric.accuracy(input=x, label = label)
            return x, acc
        else:
            return x
use_gpu = True
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')

Place(gpu:0)

In [22]:
# 定义训练过程
from visualdl import LogWriter
from paddle.optimizer.lr import CosineAnnealingDecay
from paddle.nn import BatchNorm2D
logwriter = LogWriter(logdir="./run/cifar_log/two_conv_end")
def train_Cifar_TWO2_end(model):
    # 定义优化器
    EPOCH_NUM = 20
    opt = paddle.optimizer.Adam(learning_rate=0.01, weight_decay=paddle.regularizer.L2Decay(coeff=1e-5),parameters=model.parameters())

    # 训练参数
    iter = 0
    iter2 = 0
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            model.train()
            # 数据准备
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels, (-1,1))
            labels = paddle.to_tensor(labels)

            # 前向
            predict, acc = model(images, labels)

            # 损失
            loss = F.cross_entropy(predict, labels)
            avg_loss = paddle.mean(loss)

            # 可以记录
            if batch_id%150 ==  0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(), acc.numpy()))
                logwriter.add_scalar(tag = 'train/acc', step = iter, value=acc.numpy())
                logwriter.add_scalar(tag = 'train/loss' , step = iter, value=avg_loss.numpy())
                iter+=150
            # 反向传播
            avg_loss.backward()
            opt.step()
            opt.clear_grad()

        total_test_accuracy=0
        for batch_id, data in enumerate(test_loader()):
            model.eval()
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels,(-1,1))
            labels = paddle.to_tensor(labels)

            # 预测
            predicts, acc = model(images, labels)
            loss = F.cross_entropy(predicts, labels)
            avg_loss= paddle.mean(loss)

            total_test_accuracy += acc.numpy()

            if batch_id % 100 == 0:
                logwriter.add_scalar(tag='test/acc', step=iter2, value=acc.numpy())
                logwriter.add_scalar(tag='test/loss', step=iter2, value=avg_loss.numpy())
                iter2+=100
        
        total_test_accuracy /= len(test_loader())
        logwriter.add_scalar(tag='test/total_acc', step=epoch_id, value=total_test_accuracy)

In [23]:
mdoel = Cifar_TWO_CONV2_norm_end()
train_Cifar_TWO2_end(model)

epoch: 0, batch: 0, loss is: [0.5429214], acc is [0.84375]
epoch: 0, batch: 150, loss is: [0.45616698], acc is [0.828125]
epoch: 0, batch: 300, loss is: [0.7704087], acc is [0.71875]
epoch: 0, batch: 450, loss is: [0.9242352], acc is [0.703125]
epoch: 0, batch: 600, loss is: [0.55163693], acc is [0.765625]
epoch: 0, batch: 750, loss is: [0.69744873], acc is [0.703125]
epoch: 1, batch: 0, loss is: [0.5685594], acc is [0.796875]
epoch: 1, batch: 150, loss is: [0.64746255], acc is [0.78125]
epoch: 1, batch: 300, loss is: [0.9464253], acc is [0.6875]
epoch: 1, batch: 450, loss is: [0.5504892], acc is [0.78125]
epoch: 1, batch: 600, loss is: [0.7924937], acc is [0.734375]
epoch: 1, batch: 750, loss is: [0.75680757], acc is [0.78125]
epoch: 2, batch: 0, loss is: [0.6888693], acc is [0.671875]
epoch: 2, batch: 150, loss is: [0.7400059], acc is [0.6875]
epoch: 2, batch: 300, loss is: [0.803023], acc is [0.6875]
epoch: 2, batch: 450, loss is: [0.74006987], acc is [0.75]
epoch: 2, batch: 600, lo

In [41]:
# 定义模型结构
# 把以上的都综合在一起

import paddle.nn.functional as F
from paddle.nn import Conv2D, MaxPool2D, Linear, BatchNorm2D

class Cifar_TWO_CONV2_lenet_3(paddle.nn.Layer):
    def __init__(self):
        super(Cifar_TWO_CONV2_lenet_3, self).__init__()

        # 定义一层卷积层
        self.conv1 = Conv2D(in_channels=3, out_channels=32, kernel_size=5, stride=1, padding=2)
        # 定义批归一化层
        self.bn1 = BatchNorm2D(32)
        # 定义最大池化
        self.max_pool1 = MaxPool2D(kernel_size=2, stride=2)
        # 定义第二层卷积层
        self.conv2 = Conv2D(in_channels=32, out_channels=64, kernel_size=5, stride=1, padding=2)
        # 第二个批归一化层
        self.bn2 = BatchNorm2D(64)
        # 定义最大池化
        self.max_pool2 = MaxPool2D(kernel_size=2, stride=2)
        # 再增加一层卷积
        self.conv3 = Conv2D(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.bn3 = BatchNorm2D(64)

        # 这样输出的就是20*16*16,定义一个全连接
        self.fc1 = Linear(in_features=4096, out_features=10)
        self.fc3 = Linear(in_features=1024, out_features=10)

    def forward(self, inputs, label, mode='upscale_in_train'):
        x = self.conv1(inputs)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.max_pool1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.max_pool2(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = F.relu(x)
        x = paddle.reshape(x, [x.shape[0], -1])
        x = paddle.to_tensor(x)
        x = self.fc1(x)
        if label is not None:
            acc = paddle.metric.accuracy(input=x, label = label)
            return x, acc
        else:
            return x
use_gpu = True
paddle.set_device('gpu:0') if use_gpu else paddle.set_device('cpu')

Place(gpu:0)

In [42]:
# 定义训练过程
from visualdl import LogWriter
from paddle.optimizer.lr import CosineAnnealingDecay
from paddle.nn import BatchNorm2D
logwriter = LogWriter(logdir="./run/cifar_log/two_conv_new_lenet7")
def train_Cifar_TWO2_lenet(model):
    # 定义优化器
    EPOCH_NUM = 15
    opt = paddle.optimizer.Adam(learning_rate=0.005, weight_decay=paddle.regularizer.L2Decay(coeff=1e-5),parameters=model.parameters())

    # 训练参数
    iter = 0
    iter2 = 0
    for epoch_id in range(EPOCH_NUM):
        for batch_id, data in enumerate(train_loader()):
            model.train()
            # 数据准备
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels, (-1,1))
            labels = paddle.to_tensor(labels)

            # 前向
            predict, acc = model(images, labels)

            # 损失
            loss = F.cross_entropy(predict, labels)
            avg_loss = paddle.mean(loss)

            # 可以记录
            if batch_id%200 ==  0:
                print("epoch: {}, batch: {}, loss is: {}, acc is {}".format(epoch_id, batch_id, avg_loss.numpy(), acc.numpy()))
                logwriter.add_scalar(tag = 'train/acc', step = iter, value=acc.numpy())
                logwriter.add_scalar(tag = 'train/loss' , step = iter, value=avg_loss.numpy())
                iter+=200
            # 反向传播
            avg_loss.backward()
            opt.step()
            opt.clear_grad()

        for batch_id, data in enumerate(test_loader()):
            model.eval()
            images, labels = data
            images = paddle.to_tensor(images)
            labels = paddle.reshape(labels,(-1,1))
            labels = paddle.to_tensor(labels)

            # 预测
            predicts, acc = model(images, labels)
            loss = F.cross_entropy(predicts, labels)
            avg_loss= paddle.mean(loss)

            if batch_id % 200 == 0:
                logwriter.add_scalar(tag='test/acc', step=iter2, value=acc.numpy())
                logwriter.add_scalar(tag='test/loss', step=iter2, value=avg_loss.numpy())
                iter2+=200

                

In [43]:
model = Cifar_TWO_CONV2_lenet_3()
train_Cifar_TWO2_lenet(model)

epoch: 0, batch: 0, loss is: [2.5494406], acc is [0.1328125]
epoch: 0, batch: 200, loss is: [1.546255], acc is [0.4609375]
epoch: 1, batch: 0, loss is: [1.4955535], acc is [0.5]
epoch: 1, batch: 200, loss is: [1.3238297], acc is [0.6015625]
epoch: 2, batch: 0, loss is: [1.2607361], acc is [0.59375]
epoch: 2, batch: 200, loss is: [1.2540855], acc is [0.6328125]
epoch: 3, batch: 0, loss is: [1.102382], acc is [0.640625]
epoch: 3, batch: 200, loss is: [1.1954417], acc is [0.6640625]
epoch: 4, batch: 0, loss is: [1.124525], acc is [0.640625]
epoch: 4, batch: 200, loss is: [1.3655242], acc is [0.578125]
epoch: 5, batch: 0, loss is: [1.1353847], acc is [0.625]
epoch: 5, batch: 200, loss is: [1.190679], acc is [0.625]
epoch: 6, batch: 0, loss is: [1.0680808], acc is [0.609375]
epoch: 6, batch: 200, loss is: [1.2858982], acc is [0.6328125]
epoch: 7, batch: 0, loss is: [1.3763033], acc is [0.578125]
epoch: 7, batch: 200, loss is: [1.1359574], acc is [0.640625]
epoch: 8, batch: 0, loss is: [1.12